# Make a catalog of SEDs

In [1]:
# Imports
from SEDkit import sed, spectrum, synphot, catalog
import astropy.units as q
import numpy as np
import copy
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:119: UserWarning: Extinction files should be moved to $PYSYN_CDBS/extinction for compatibility with future versions of pysynphot.
  warnings.warn('Extinction files should be moved to '
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:155: UserWarning: Extinction files not found in /grp/hst/cdbs/grid/extinction
  warnings.warn('Extinction files not found in %s' % (extdir,))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:114: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /grp/hst/cdbs/mtab/*_tmg.fits.
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:121: UserWarning: No thermal tables found, no thermal calculations can be performed. No files 

Loading BokehJS ...

First we initialize an empty catalog.

In [2]:
# Make an SED catalog
cat = catalog.SEDCatalog()

Now let's make a few SEDs

In [3]:
# Make an SED
s1 = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')
s1.add_photometry_file(resource_filename('SEDkit', 'data/M0V_photometry.txt'))
s1.make_sed()
cat.add_SED(s1)

Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)
Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)


In [4]:
# Make 30 other SEDs (I'll just change the parallax)
for i in range(10):
    s2 = copy.copy(s1)
    s2.parallax = abs(np.random.normal(loc=1.7474, scale=0.1))*q.mas, 0.1*q.mas
    s2.name += str(i)
    s2.make_sed()
    cat.add_SED(s2)

Then the (abridged) aggregated results look like this:

In [9]:
cat.results[['name','distance','distance_unc','Lbol_sun','Lbol_sun_unc','Teff','Teff_unc']]

name,distance,distance_unc,Lbol_sun,Lbol_sun_unc,Teff,Teff_unc
,pc,pc,,,K,K
object,float64,float64,object,object,float64,float64
2MASS 261832886,572.28,16.41,-1.019,0.025,3424.0,141.0
2MASS 2618328860,577.95,33.4,-1.011,0.05,3441.0,166.0
2MASS 2618328861,543.79,29.57,-1.064,0.047,3338.0,157.0
2MASS 2618328862,567.46,32.2,-1.027,0.049,3410.0,163.0
2MASS 2618328863,544.74,29.67,-1.062,0.047,3341.0,158.0
2MASS 2618328864,556.1,30.92,-1.044,0.048,3375.0,160.0
2MASS 2618328865,577.15,33.31,-1.012,0.05,3439.0,165.0
2MASS 2618328866,564.56,31.87,-1.031,0.049,3401.0,162.0


We can also plot the results by passing the `plot` method two column names.

In [10]:
show(cat.plot('distance', 'Teff'))

Or plot a CMD! You can also pass the difference of two magnitudes as a string like so.

In [11]:
show(cat.plot('2MASS.J-2MASS.Ks', 'M_2MASS.J'))

We can then retrieve the SED for a particular source like so:

In [12]:
my_sed = cat.get_SED('2MASS 261832886')
fig = my_sed.plot()

And save and load from file.

In [13]:
cat.save('/Users/jfilippazzo/Desktop/test.p')

PicklingError: Can't pickle <function ColumnDataSource.<lambda> at 0x112503730>: attribute lookup ColumnDataSource.<lambda> on bokeh.models.sources failed

In [ ]:
new = sed.SEDCatalog().load('/Users/jfilippazzo/Desktop/test.p')

The catalog can also be filtered by providing a column name and value...

In [39]:
cat.filter('name', "'2MASS 2618328861'").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 2618328861,6000.0,4000.0,598.98,35.88,1.6695005640755598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.01144652103885e+32,4.820994360627056e+31,-0.98,0.052,7.205,0.066,4.5,0.07,0.8896913720017506,0.0,3503.0,171.0,<SEDkit.sed.SED object at 0x11af7de48>,20.486,0.073,73499.064,8805.458,17.779,0.006,63786.97,7641.913,16.431,0.004,58950.656,7062.505,15.843,0.004,56841.047,6809.766,15.514,0.005,55660.67,6668.352,14.391,0.029,51631.604,6185.656,13.677,0.032,49069.936,5878.759,13.535,0.034,48560.472,5817.723,13.55,0.027,48614.289,5824.17,13.581,0.035,48725.51,5837.495,12.884,0.0,46224.834,5537.905


...or a wildcard search...

In [43]:
cat.filter('name', "*328861").results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 2618328861,6000.0,4000.0,598.98,35.88,1.6695005640755598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.01144652103885e+32,4.820994360627056e+31,-0.98,0.052,7.205,0.066,4.5,0.07,0.8896913720017506,0.0,3503.0,171.0,<SEDkit.sed.SED object at 0x11af7de48>,20.486,0.073,73499.064,8805.458,17.779,0.006,63786.97,7641.913,16.431,0.004,58950.656,7062.505,15.843,0.004,56841.047,6809.766,15.514,0.005,55660.67,6668.352,14.391,0.029,51631.604,6185.656,13.677,0.032,49069.936,5878.759,13.535,0.034,48560.472,5817.723,13.55,0.027,48614.289,5824.17,13.581,0.035,48725.51,5837.495,12.884,0.0,46224.834,5537.905


... or an argument with an operator...

In [41]:
cat.filter('logg', '>=4.5').results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0,<SEDkit.sed.SED object at 0x11a1bdac8>,20.486,0.073,67092.551,3847.73,17.779,0.006,58227.007,3339.293,16.431,0.004,53812.248,3086.108,15.843,0.004,51886.522,2975.669,15.514,0.005,50809.032,2913.875,14.391,0.029,47131.158,2702.951,13.677,0.032,44792.777,2568.846,13.535,0.034,44327.72,2542.176,13.55,0.027,44376.846,2544.993,13.581,0.035,44478.372,2550.815,12.884,0.0,42195.667,2419.902
2MASS 2618328860,6000.0,4000.0,572.64,32.79,1.7463009673899073,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.6663990004665696e+32,4.2133096135621395e+31,-1.019,0.05,7.303,0.063,4.5,0.07,0.8896913720017506,0.0,3425.0,164.0,<SEDkit.sed.SED object at 0x11afe1ef0>,20.486,0.073,67176.988,7693.259,17.779,0.006,58300.287,6676.678,16.431,0.004,53879.972,6170.454,15.843,0.004,51951.822,5949.638,15.514,0.005,50872.977,5826.086,14.391,0.029,47190.474,5404.358,13.677,0.032,44849.149,5136.224,13.535,0.034,44383.508,5082.898,13.55,0.027,44432.695,5088.531,13.581,0.035,44534.349,5100.173,12.884,0.0,42248.771,4838.423
2MASS 2618328861,6000.0,4000.0,598.98,35.88,1.6695005640755598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.01144652103885e+32,4.820994360627056e+31,-0.98,0.052,7.205,0.066,4.5,0.07,0.8896913720017506,0.0,3503.0,171.0,<SEDkit.sed.SED object at 0x11af7de48>,20.486,0.073,73499.064,8805.458,17.779,0.006,63786.97,7641.913,16.431,0.004,58950.656,7062.505,15.843,0.004,56841.047,6809.766,15.514,0.005,55660.67,6668.352,14.391,0.029,51631.604,6185.656,13.677,0.032,49069.936,5878.759,13.535,0.034,48560.472,5817.723,13.55,0.027,48614.289,5824.17,13.581,0.035,48725.51,5837.495,12.884,0.0,46224.834,5537.905
2MASS 2618328862,6000.0,4000.0,569.45,32.43,1.7560932438236598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.62566403281865e+32,4.143994293920703e+31,-1.024,0.05,7.315,0.063,4.5,0.07,0.8896913720017506,0.0,3416.0,164.0,<SEDkit.sed.SED object at 0x11afbc358>,20.486,0.073,66430.629,7566.408,17.779,0.006,57652.55,6566.59,16.431,0.004,53281.346,6068.712,15.843,0.004,51374.619,5851.537,15.514,0.005,50307.76,5730.023,14.391,0.029,46666.171,5315.248,13.677,0.032,44350.86,5051.

... or even a tuple of arguments...

In [44]:
cat.filter('M_SDSS.r', ('>50000','<60000')).results

name,age,age_unc,distance,distance_unc,parallax,parallax_unc,radius,radius_unc,spectral_type,spectral_type_unc,membership,fbol,fbol_unc,mbol,mbol_unc,Lbol,Lbol_unc,Lbol_sun,Lbol_sun_unc,Mbol,Mbol_unc,logg,logg_unc,mass,mass_unc,Teff,Teff_unc,SED,SDSS.u,SDSS.u_unc,M_SDSS.u,M_SDSS.u_unc,SDSS.g,SDSS.g_unc,M_SDSS.g,M_SDSS.g_unc,SDSS.r,SDSS.r_unc,M_SDSS.r,M_SDSS.r_unc,SDSS.i,SDSS.i_unc,M_SDSS.i,M_SDSS.i_unc,SDSS.z,SDSS.z_unc,M_SDSS.z,M_SDSS.z_unc,2MASS.J,2MASS.J_unc,M_2MASS.J,M_2MASS.J_unc,2MASS.H,2MASS.H_unc,M_2MASS.H,M_2MASS.H_unc,2MASS.Ks,2MASS.Ks_unc,M_2MASS.Ks,M_2MASS.Ks_unc,WISE.W1,WISE.W1_unc,M_WISE.W1,M_WISE.W1_unc,WISE.W2,WISE.W2_unc,M_WISE.W2,M_WISE.W2_unc,WISE.W3,WISE.W3_unc,M_WISE.W3,M_WISE.W3_unc
,Myr,Myr,pc,pc,mas,mas,6.957e+08 m,,,,,erg / (cm2 s),erg / (cm2 s),,,erg / s,erg / s,,,,,,,solMass,solMass,K,K,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,float64,float64,object,object,float64,float64,object,object,object,object,object,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2MASS 261832886,6000.0,4000.0,572.28,16.41,1.7474,0.0501,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.66179055973561e+32,2.1287295997847215e+31,-1.019,0.025,7.304,0.033,4.5,0.07,0.8896913720017506,0.0,3424.0,141.0,<SEDkit.sed.SED object at 0x11a1bdac8>,20.486,0.073,67092.551,3847.73,17.779,0.006,58227.007,3339.293,16.431,0.004,53812.248,3086.108,15.843,0.004,51886.522,2975.669,15.514,0.005,50809.032,2913.875,14.391,0.029,47131.158,2702.951,13.677,0.032,44792.777,2568.846,13.535,0.034,44327.72,2542.176,13.55,0.027,44376.846,2544.993,13.581,0.035,44478.372,2550.815,12.884,0.0,42195.667,2419.902
2MASS 2618328860,6000.0,4000.0,572.64,32.79,1.7463009673899073,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.6663990004665696e+32,4.2133096135621395e+31,-1.019,0.05,7.303,0.063,4.5,0.07,0.8896913720017506,0.0,3425.0,164.0,<SEDkit.sed.SED object at 0x11afe1ef0>,20.486,0.073,67176.988,7693.259,17.779,0.006,58300.287,6676.678,16.431,0.004,53879.972,6170.454,15.843,0.004,51951.822,5949.638,15.514,0.005,50872.977,5826.086,14.391,0.029,47190.474,5404.358,13.677,0.032,44849.149,5136.224,13.535,0.034,44383.508,5082.898,13.55,0.027,44432.695,5088.531,13.581,0.035,44534.349,5100.173,12.884,0.0,42248.771,4838.423
2MASS 2618328861,6000.0,4000.0,598.98,35.88,1.6695005640755598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,4.01144652103885e+32,4.820994360627056e+31,-0.98,0.052,7.205,0.066,4.5,0.07,0.8896913720017506,0.0,3503.0,171.0,<SEDkit.sed.SED object at 0x11af7de48>,20.486,0.073,73499.064,8805.458,17.779,0.006,63786.97,7641.913,16.431,0.004,58950.656,7062.505,15.843,0.004,56841.047,6809.766,15.514,0.005,55660.67,6668.352,14.391,0.029,51631.604,6185.656,13.677,0.032,49069.936,5878.759,13.535,0.034,48560.472,5817.723,13.55,0.027,48614.289,5824.17,13.581,0.035,48725.51,5837.495,12.884,0.0,46224.834,5537.905
2MASS 2618328862,6000.0,4000.0,569.45,32.43,1.7560932438236598,0.1,0.8786209573091851,0.06782337214316517,60.0,0.5,None,9.344700986227546e-12,8.891778300076678e-14,16.092,0.01,3.62566403281865e+32,4.143994293920703e+31,-1.024,0.05,7.315,0.063,4.5,0.07,0.8896913720017506,0.0,3416.0,164.0,<SEDkit.sed.SED object at 0x11afbc358>,20.486,0.073,66430.629,7566.408,17.779,0.006,57652.55,6566.59,16.431,0.004,53281.346,6068.712,15.843,0.004,51374.619,5851.537,15.514,0.005,50307.76,5730.023,14.391,0.029,46666.171,5315.248,13.677,0.032,44350.86,5051.